# --- Day 8: Playground ---


In [19]:
# --- Support Functions ---
# This section contains support functions used by the main code.

def read_input(file_path):
    with open(file_path, 'r') as file:
        return file.read().splitlines()

def distance(p0:tuple[int, int, int], p1:tuple[int, int, int]) -> float:
    return ((p0[0]-p1[0])**2 + (p0[1]-p1[1])**2 + (p0[2]-p1[2])**2)

class  Connections:
    """ 
    A class to take track of junction boxes connections.
    The array connection for each box (x) track the box to wich is connected. If is coneccted to himself
    means no connection
    The array sizes keep track of all other boxes that are connected to this box starting with 1 that
    means a self connection
    """
    def __init__(self, size: int) -> None:
        self.connections = list(range(size)) # start with all boxes connected to themself = no connection
        self.sizes = [1] * size                 # all sizes are 1 = no connection

    def find(self, x:int) -> int:
        """
        Returns the root of the connection for the x boxes
        If the x is not connected returns x
        While following the connections try to compress the path pointing to the connection of the connected box.
        At the end all points to the root
        """
        while self.connections[x] != x:
            self.connections[x] = self.connections[self.connections[x]]
            x = self.connections[x]
        return x

    def connect(self, b1: int, b2: int) -> bool:
        # return True if the connection form a single circuit with all boxes
        root1, root2 = self.find(b1), self.find(b2)
        if root1 == root2:
            return  # both box are part of the same connection
        # merge the connection attaching the smaller connection to the larger
        if self.sizes[root1] < self.sizes[root2]:
            root1, root2 = root2, root1     # root1 is the root of larger connection
        self.connections[root2] = root1
        self.sizes[root1] += self.sizes[root2]
        if self.sizes[root1] == len(self.sizes):
            return True
        else:
            return False




In [20]:
# --- Part ONE ---

input = read_input('input.txt')

MAX_CONNECTIONS = 1000

result = 0

jboxes = [(int(x), int(y), int(z)) for x,y,z in (line.split(',') for line in input)]

# calculate all straightline distances
distances = [
    (distance(jboxes[i], jboxes[j]), i, j)
    for i in range(len(jboxes))
    for j in range(i+1, len(jboxes))
]

distances.sort(key=lambda x: x[0])  # sort by distances

# create the first MAX_CONNECTION connections (or for all distances if less)

connections = Connections(len(jboxes))

for c in range(min(MAX_CONNECTIONS, len(distances))):
    _, b1, b2 = distances[c]
    connections.connect(b1, b2)

#for i in range(len(connections.connections)):
#    print(f" box: {i:3}, root: {connections.connections[i]:3}, size: {connections.sizes[i]}")

# calculate the sizes

conn_sizes = {}

for i in range(len(jboxes)):
    #extract only root
    if connections.connections[i] == i:
        conn_sizes[i] = connections.sizes[i]

# print (conn_sizes)

sizes = sorted(conn_sizes.values(), reverse = True)

# print (sizes)

result = sizes[0] * sizes[1] * sizes[2]

print("Part ONE:", result)

Part ONE: 352584


In [24]:
# --- Part TWO ---
input = read_input('test1.txt')

result = 0

jboxes = [(int(x), int(y), int(z)) for x,y,z in (line.split(',') for line in input)]

distances = [
    (distance(jboxes[i], jboxes[j]), i, j)
    for i in range(len(jboxes))
    for j in range(i+1, len(jboxes))
]

distances.sort(key=lambda x: x[0])  # sort by distances



connections = Connections(len(jboxes))

for c in range(len(distances)):
    _, b1, b2 = distances[c]
    if connections.connect(b1, b2):
        result = jboxes[b1][0] * jboxes[b2][0]
        break


# calculate the sizes


print("Part TWO:", result)

Part TWO: 25272


In [25]:
# easter egg

!pip install plotly

try:
    import plotly.graph_objects as go
except Exception as e:
    print('Plotly non disponibile:', e)
    raise

# prepare union-find and plotting lists
connections_viz = Connections(len(jboxes))
edge_x, edge_y, edge_z = [], [], []
highlight_edge = None
stopped = False

for _, b1, b2 in distances:
    res = connections_viz.connect(b1, b2)
    # res is None when b1 and b2 already in same component -> discard
    if res is None:
        continue
    # add this edge to visualization
    x0, y0, z0 = jboxes[b1]
    x1, y1, z1 = jboxes[b2]
    edge_x += [x0, x1, None]
    edge_y += [y0, y1, None]
    edge_z += [z0, z1, None]
    if res is True:
        # this merge created a single circuit containing all boxes -> stop
        highlight_edge = (x0, y0, z0, x1, y1, z1)
        stopped = True
        break

# node coordinates and component colors
xs = [p[0] for p in jboxes]
ys = [p[1] for p in jboxes]
zs = [p[2] for p in jboxes]

node_text = [f"{i}: {pt}" for i, pt in enumerate(jboxes)]

edge_trace = go.Scatter3d(
    x=edge_x, y=edge_y, z=edge_z,
    mode='lines',
    line=dict(color='gray', width=2),
    hoverinfo='none'
)

node_trace = go.Scatter3d(
    x=xs, y=ys, z=zs,
    mode='markers+text',
    marker=dict(size=4, color='red'),
    text=[str(i) for i in range(len(jboxes))],
    textposition='top center',
    hovertext=node_text,
    hoverinfo='text'
)

fig = go.Figure(data=[edge_trace, node_trace])
fig.update_layout(
    scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'),
    margin=dict(l=0, r=0, b=0, t=0)
)